9. Epileptic Seizure Recognition dataset: Epileptic.csv

Epileptic.csv (7.6MB): 11500 samples
the last column is label includes 5 classes

Attribute Information:

The original dataset from the reference consists of 5 different folders, each with 100 files, with each file representing a single subject/person. Each file is a recording of brain activity for 23.6 seconds. The corresponding time-series is sampled into 4097 data points. Each data point is the value of the EEG recording at a different point in time. So we have total 500 individuals with each has 4097 data points for 23.5 seconds.

We divided and shuffled every 4097 data points into 23 chunks, each chunk contains 178 data points for 1 second, and each data point is the value of the EEG recording at a different point in time. So now we have 23 x 500 = 11500 pieces of information(row), each information contains 178 data points for 1 second(column), the last column represents the label y {1,2,3,4,5}.

The response variable is y in column 179, the Explanatory variables X1, X2, ..., X178

y contains the category of the 178-dimensional input vector. Specifically y in {1, 2, 3, 4, 5}:
5 - eyes open, means when they were recording the EEG signal of the brain the patient had their eyes open
4 - eyes closed, means when they were recording the EEG signal the patient had their eyes closed
3 - Yes they identify where the region of the tumor was in the brain and recording the EEG activity from the healthy brain area
2 - They recorder the EEG from the area where the tumor was located
1 - Recording of seizure activity

All subjects falling in classes 2, 3, 4, and 5 are subjects who did not have epileptic seizure. Only subjects in class 1 have epileptic seizure. Our motivation for creating this version of the data was to simplify access to the data via the creation of a .csv version of it. Although there are 5 classes most authors have done binary classification, namely class 1 (Epileptic seizure) against the rest.

https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
data = pd.read_csv('./Epileptic.csv').drop('Unnamed: 0',axis=1)
data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,...,15,16,12,5,-1,-18,-37,-47,-48,2
11496,-47,-11,28,77,141,211,246,240,193,136,...,-65,-33,-7,14,27,48,77,117,170,1
11497,14,6,-13,-16,10,26,27,-9,4,14,...,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,-40,-25,-9,-12,-2,12,7,19,22,29,...,121,135,148,143,116,86,68,59,55,3


In [3]:
X = data.drop('y',axis=1)
y = data['y']

In [18]:
# Use PCA to reduce dimension to 32 dimensions
from sklearn.decomposition import PCA

n_components = 128
X_pca = PCA(n_components = n_components).fit_transform(X)

print('X_pca.shape',X_pca.shape)

X_pca.shape (11500, 128)


In [5]:
X_pca

array([[-9.13534490e+01, -1.84251674e+02,  1.15333455e+02, ...,
        -3.13214247e+00,  7.83319220e+00,  1.15195497e+00],
       [-3.13252394e+02, -4.74852194e+02,  9.66017585e+02, ...,
        -5.82656941e+00,  1.41557118e+01,  2.89133532e+01],
       [ 9.10749115e+01, -4.22180073e+01,  1.21638616e+02, ...,
        -2.87463085e+00, -7.70440284e-02,  4.33385897e+00],
       ...,
       [-5.12943417e+01, -2.20869310e+01, -3.64318058e+01, ...,
         9.11902746e-01, -5.20160205e+00, -7.23813019e+00],
       [-4.54683137e+01, -1.76955508e+02,  1.10652636e+02, ...,
         6.01965307e+00,  2.00791215e+00,  6.16757570e+00],
       [ 1.94144101e+01,  4.80783586e+01,  4.64953714e+01, ...,
         1.00636846e-01, -1.10717306e+00,  1.68054475e+00]])

In [6]:
# Mark categories from 0
y = y-1
y.value_counts()

4    2300
3    2300
2    2300
1    2300
0    2300
Name: y, dtype: int64

In [7]:
# Divide training test set
from sklearn.model_selection import train_test_split

x_pca_train,x_pca_test,y_train,y_test = train_test_split(X_pca,y,test_size=0.2,random_state=11)

In [8]:
# One-Hot 
import keras

y_train = keras.utils.to_categorical(y_train, 5)
y_test = keras.utils.to_categorical(y_test, 5)

Using TensorFlow backend.


In [9]:
#  Build a fully connected network
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(128,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               66048     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 2565      
Total params: 331,269
Trainable params: 331,269
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_pca_train, y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_pca_test, y_test))

Train on 9200 samples, validate on 2300 samples
Epoch 1/10
9200/9200 [==============================] - 1s 160us/step - loss: 18.7240 - accuracy: 0.3523 - val_loss: 3.5765 - val_accuracy: 0.3991
Epoch 2/10
9200/9200 [==============================] - 1s 128us/step - loss: 3.8052 - accuracy: 0.4283 - val_loss: 2.6890 - val_accuracy: 0.4439
Epoch 3/10
9200/9200 [==============================] - 1s 130us/step - loss: 3.0828 - accuracy: 0.4672 - val_loss: 1.9337 - val_accuracy: 0.4639
Epoch 4/10
9200/9200 [==============================] - 1s 127us/step - loss: 2.4258 - accuracy: 0.4915 - val_loss: 1.9024 - val_accuracy: 0.4848
Epoch 5/10
9200/9200 [==============================] - 1s 140us/step - loss: 2.2539 - accuracy: 0.5132 - val_loss: 1.5757 - val_accuracy: 0.5191
Epoch 6/10
9200/9200 [==============================] - 1s 148us/step - loss: 1.8964 - accuracy: 0.5346 - val_loss: 1.5748 - val_accuracy: 0.5170
Epoch 7/10
9200/9200 [==============================] - 2s 170us/step - los

Because the data is divided into five categories, there are many interference factors for the prediction results, so the prediction effect is not very good.


In [11]:
score = model.evaluate(x_pca_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.3362522888183594
Test accuracy: 0.5839130282402039


### Try a PCA + GBDT model

In [12]:
import pandas as pd
data = pd.read_csv('Epileptic.csv').drop('Unnamed: 0',axis=1)
data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,...,15,16,12,5,-1,-18,-37,-47,-48,2
11496,-47,-11,28,77,141,211,246,240,193,136,...,-65,-33,-7,14,27,48,77,117,170,1
11497,14,6,-13,-16,10,26,27,-9,4,14,...,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,-40,-25,-9,-12,-2,12,7,19,22,29,...,121,135,148,143,116,86,68,59,55,3


In [13]:
X = data.drop('y',axis=1)
y = data['y']

In [14]:
# Use PCA to reduce to 128 dimensions.
from sklearn.decomposition import PCA

n_components = 128
X_pca = PCA(n_components = n_components).fit_transform(X)

print('X_pca.shape',X_pca.shape)

X_pca.shape (11500, 128)


In [15]:
# Try to divide the data set
from sklearn.model_selection import train_test_split

x_pca_train,x_pca_test,y_train,y_test = train_test_split(X_pca,y,test_size=0.2,random_state=11)

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

gbdt = GradientBoostingClassifier()

gbdt.fit(x_pca_train,y_train)

y_pred = gbdt.predict(x_pca_test)

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_pred,y_test))
accuracy_score(y_pred,y_test)

[[439  25  10   1   0]
 [  6 218 157   9   7]
 [  2 161 243   2  13]
 [  5  15  11 362  38]
 [  1  31  40  77 427]]


0.7343478260869565